We want the neural net to be given the same size volume at each episode.



First, we upsample the SWC files to make sure to get rid of huge inter-node distances, which can cause problems when trying to render a volume.

Before running the `resample_swc` script, this are the steps that Yang took:

He went to 'v3d_tools/release_plugins/v3d_plugins/resample_swc' and ran `qmake` and then `make`. He made some symbolic links to the main `v3d_external` folder.

Then, to run the `resample_swc` script, simply run `/path/to/v3d_external/bin/vaa3d -x resample_swc -f resample_swc -i <input_file> -o <output_file>`.

We need to get the resample_swc file directory

we check to see if the branch is at least as big as the target volume before we sample nodes from it.

I tried minimum dimension cut-off of 25, but no branches met that criteria.

In [7]:
from swc_io import get_fnames_and_abspath_from_dir, swc_to_dframe, resample_swc, dframe_to_swc
import numpy as np
import pandas as pd


BRANCHES_DIR = "../data/03_human_branches_splitted"
UPSAMPLED_BRANCHES_DIR = "../data/04_human_branches_filtered_upsampled"

fnames, abs_paths = get_fnames_and_abspath_from_dir(BRANCHES_DIR)
total_dim = 0
for i in range(len(fnames)):
    # upsample the SWC
    upsampled_fpath = resample_swc(fnames, abs_paths[i])
  

TypeError: join() argument must be str or bytes, not 'list'

In [ ]:
# tested size 25, but no branches met this criteria
TARGET_DIM_SIZE = 10.  # size in pixels for all x,y,z dims
half_size = TARGET_DIM_SIZE / 2
#min_swc_dim_span = TARGET_DIM_SIZE + 5  # some leeway to make things easy
padding = TARGET_DIM_SIZE//2

print("filtering branches that have any dimension smaller than ", TARGET_DIM_SIZE)

fnames, abs_paths = get_fnames_and_abspath_from_dir(UPSAMPLED_BRANCHES_DIR)
total_dim = 0
for i in range(len(fnames)):
    
        # replace nodes with our upsampled nodes
        nodes = swc_to_dframe(upsampled_fpath)
    
    if i % 1000 == 0 and i != 0: print("testing branch #", i, "avg span: ", total_dim/i/3)
    nodes = swc_to_dframe(abs_paths[i])
    assert(isinstance(nodes, pd.DataFrame))
    
    x_span = nodes.x.max() - nodes.x.min()
    y_span = nodes.y.max() - nodes.y.min()
    z_span = nodes.z.max() - nodes.z.min()
    
    total_dim = total_dim + x_span + y_span + z_span
    
    dim_size_good = np.array([x_span, y_span, z_span]) > TARGET_DIM_SIZE
    if z_span > TARGET_DIM_SIZE:   print("found z bigger than {}: {}".format(int(TARGET_DIM_SIZE), z_span))
    if np.all(dim_size_good):  # our SWC meets our criteria
        print("{} is big enough".format(i))
        
        # upsample the SWC
        upsampled_fpath = resample_swc(fnames, abs_paths[i])
        # replace nodes with our upsampled nodes
        nodes = swc_to_dframe(upsampled_fpath)
        
        # find nodes far enough away from bounds
        central_nodes = nodes.loc[(nodes.x.between(nodes.x.min()+padding,nodes.x.max()+padding)) &
                              (nodes.y.between(nodes.y.min()+padding,nodes.y.max()+padding)) &
                              (nodes.z.between(nodes.z.min()+padding,nodes.z.max()+padding))]
        
        if len(central_nodes) > 0:
            # sample a central node
            center_node = central_nodes.sample(n=1)
            # grab all nodes in neighborhood of center_node
            sample = nodes.loc[(nodes.x.between(center_node.x.item()-half_size, center_node.x.item()+half_size)) &
                              (nodes.y.between(center_node.y.item()-half_size, center_node.y.item()+half_size)) &
                              (nodes.z.between(center_node.z.item()-half_size, center_node.z.item()+half_size))]
            
            # save sample to swc
            print("saving ", fnames[i])
            dframe_to_swc(fnames[i], sample)
    else:  # SWC does not have large enough dims
        #print("dims too small ", x_span, y_span, z_span) 
        pass
    
    

Next, we want to only randomly sample from nodes sufficiently far away from the boundary

In [15]:
# sample from the SWC
# first, we won't pick nodes too close to the edges
#filtered_nodes = np.where()
#print("dim sizes are good")

for swc_file in good_swc_paths[:1]: # TODO remove slice
    nodes = good_swcs[swc_file]
    good_nodes = 